# Experiments with applying openai to building itineraries and blog removal os sensitive information

In [ ]:
import sys
import os

In [ ]:
# pip install pyarrow

In [ ]:
import json
import pandas as pd
from IPython.display import display, Markdown
from importlib import reload

In [ ]:
import openai
from langchain.document_loaders import PyPDFLoader
from langchain import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']

## Using various models

In [ ]:
m35 = "gpt-3.5-turbo"
m35_latest = 'gpt-3.5-turbo-1106'
m4 = 'gpt-4-1106-preview'
client = openai.OpenAI()

In [ ]:
def ask_openai ( prompt, model = m35_latest):
    response = client.chat.completions.create(
        model=model,
        
        messages=[
            {"role": "system", "content": "You are a travel agent"},
            {"role": "user", "content": prompt}
        ],
        temperature = 0
    )
    return response.choices[0].message.content

## Creating a simple prompts

In [ ]:
p_itinerary = '''
Act as a travel agent and help me to plan my trip. 
I will drive from Mountain View, CA to Palm Springs, CA on October 20th. I want to arrive to Palm Springs by 2 PM. 
I will stay in Palm Springs for two days.
On the way back I'd like stay overnight somewhere in between from Palm Springs and Mountain View. It should be a locations where I can
visit local winderies. Suggest routes, lunch times
and locations.
Present results in json format and include text recommendaitons for each day.
'''

In [ ]:
answer = ask_openai(p_itinerary, m35)
print (answer)

In [ ]:
answer = ask_openai(p_itinerary, m35_latest)
print (answer)

In [ ]:
answer = ask_openai(p_itinerary, m4)
print (answer)

## A request on lunch recomendations. Using the latest gpt-4 model

In [ ]:
p_lunch = ''' 
act as a food connoiseur and suggest a place for lunch in Palm Springs, CA on budget. I prefer mexican cousine. Present resulat in json format
using following keys: restaurant_name, location, approximate_lunch_price, text.
'''
answer = ask_openai(p_lunch, m4)
print (answer)

In [ ]:
answer = ask_openai(p_lunch, m35_latest)
print (answer)

## Removal of sensitive information. The text below contains personal info 

In [ ]:
t_2023_10_26 = '''
We went to Dominick’s Steakhouse for Alex’s birthday dinner. 
There was a small hiccup on the way when Vanessa discovered that her car’s tires did not have enough pressure. 
Alex was not able to find a working air compressor at a gas station. Luckily, Vanessa and Gary had their portable ones. 
We were late but Vanessa warned them over the phone. The restaurant staff was very accomodating for our tardiness.
They sat  us at a table close to the aisle. Then when Alex mentioned that when our waiter, 
Nate, stepped at a certain spot the whole table shook. 
Suprisingly, almost immediately, a manager came and moved us to a better table. 
The waiter gave us a menu and a wine list. The wine list had wines with prices from $275 to $4000. 
I panicked initially but then noticed on the other side of the menu a list of normally priced wines and wines by the glass. 

Lesha ordered appetizers, which included crabs on ice, devil’s eggs and something else. Crabs were very good: fresh and deliious.
I ordered a vodka martini dirty with juice but no blue cheese on ice. 
It was good and lasted long. I ordered a beet salad - nothing spectacular, and a petit filet mignon, medium rare. 
This one was not petit: 8 oz piece was more than enough. Also came the sides, which were very good, especially sweet potatoes. 
I ordered a glass of Oregon Pinot Noir, which paired nicely with the filet mignon. 
The whole dinner came at $1200 for 7 people. Costly, but worth it.
'''

### Prompt to remove the sensitive info

In [ ]:
p_leave_restaurant = f'''
Leave only references to the dinner at the restaurant, meals, sentiments and costs from the text separated by '-'.
 -{t_2023_10_26} - Remove personal references and correct the grammer. Additionally, deduce the sentiment and rate i on the scale
 from 1 to 5. Present result in json format using the following scheme: 
  'response' : response; 'sentiment':sentiment;
 '''

### Result

In [ ]:
answer = ask_openai(p_leave_restaurant)
print(answer)

In [ ]:
answer = ask_openai(p_leave_restaurant, m4)
print(answer)

## Itenerary Prompt created by chatgpt and adapted

In [ ]:
itinerary_template = '''
Generate a detailed travel itinerary for a trip to {Destination}. 
Include recommendations for accommodation, transportation, activities, and dining options. 
Consider the preferences of a traveler who enjoys {Preferences}, 
The trip should start on {StartDate} and end on {EndDate}. 
Please provide a day-by-day breakdown of the itinerary, including estimated costs and any additional tips or suggestions.
Try to fit this trip in my budget of {Budget} dollars.

Additionally, for each day of the itinerary, include a JSON object with the following structure:

  "day": 1,
  "date": "[Date]",
  "accommodation": "[Accommodation Recommendation]",
  "transportation": "[Transportation Recommendation]",
  "activities": ["Activity 1", "Activity 2", ...],
  "dining_options": ["Dining Option 1", "Dining Option 2", ...]

'''
def itinerary_prompt(Destination, Preferences, StartDate, EndDate, Budget):
    return itinerary_template.format(Destination = Destination, Preferences = Preferences, StartDate = StartDate, EndDate = EndDate, Budget = Budget)

### Create a prompt for a trip to NYC

In [ ]:
p_NYC = itinerary_prompt('New York City', 'Museums, dining, walking around the city', 'April 29', 'May 4', 2000)

In [ ]:
print (p_NYC)

### Ask OpenAI

In [ ]:
answer = ask_openai(p_NYC, m4)
print(answer)

### Change the dates

In [ ]:
p_NYC = itinerary_prompt('New York City', 'Museums, dining, walking around the city', 'April 29', 'May 1', 2000)
answer = ask_openai(p_NYC, m4)
print(answer)

### Plan a trip to Mountain View

In [ ]:
p_SF = itinerary_prompt('Mountain View, CA', 'Museums, dining, hiking', 'April 29', 'April 30', 2000)
answer = ask_openai(p_SF, m4)
print(answer)

### One day trip to Carmel

In [ ]:
p_carmel = itinerary_prompt('Carmel by the sea, CA', 'nature, dining, hiking', 'April 29', 'April 29', 2000)
answer = ask_openai(p_carmel, m4)
print(answer)

In [ ]:
p_japan = '''I want to make a trip to Japan and visit Tokyo, Hakone and Kyoto. Duration of the trip is 10 days.
Please, suggest the best time of the year and durations of the stay in each place as well as places to visit.'''
answer = ask_openai(p_japan, m4)
print(answer)